In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_mistralai import  ChatMistralAI
import os

llm = ChatMistralAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPEN_ROUTER_API_KEY"),
    model = 'mistralai/mistral-nemo:free',
    verbose=True
)

In [3]:
llm.invoke("Hello")


AIMessage(content=' Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 7, 'completion_tokens': 10, 'total_tokens': 17}, 'model_name': 'mistralai/mistral-nemo:free', 'model': 'mistralai/mistral-nemo:free', 'finish_reason': 'stop'}, id='run--84a6efd2-351f-4c32-a411-1c84bd533476-0', usage_metadata={'input_tokens': 7, 'output_tokens': 10, 'total_tokens': 17})

In [6]:
from langchain_core.tools import StructuredTool

In [ ]:
def getweather(city :str):
    """get the weather information for a location"""
    

In [7]:
import requests
from pydantic import BaseModel

class WeatherInfoTool(BaseModel):
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "http://api.openweathermap.org/data/2.5/weather"

    def get_weather(self, city):
        params = {
            "q": city,
            "appid": self.api_key,
            "units": "metric"
        }
        response = requests.get(self.base_url, params=params)
        weather_data = response.json()
        return weather_data

    def print_weather(self, city):
        weather_data = self.get_weather(city)
        print(f"Weather in {city}:")
        print(f"Temperature: {weather_data['main']['temp']}°C")
        print(f"Humidity: {weather_data['main']['humidity']}%")
        print(f"Description: {weather_data['weather'][0]['description']}")


In [ ]:
weather_tool = StructuredTool.from_function(
    func=getweather,
    name="weather tool",
    description="get the weather information for a location",
    args_schema=WeatherInfoTool
)